In [1]:
import os
import io
import matlab.engine
import sys
sys.path.append("../")
import od.vars as GV
import od.engine
from od.network.allocator import ResourceAllocatorNomaApprox



In [2]:
print(matlab.engine.find_matlab())
os.getcwd()

('MATLAB_771504',)


'/home/OD/repos/ProjectSumo/ipynb'

In [3]:
MATLAB_ENG = matlab.engine.connect_matlab()
od.engine.MATLAB_ENG = MATLAB_ENG

In [4]:
if True:
    MATLAB_ENG.addpath(os.getcwd() + "/../matlab/")
    MATLAB_ENG.addpath(os.getcwd() + "/../matlab/SelectCQI_bySNR/")
    MATLAB_ENG.addpath(os.getcwd() + "/../matlab/PlannerV1/")
print("Done")

Done


In [5]:

# QoS_GP_CONF = [
# 	[
# 		{'gid': 0.0, 'qos': 0.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 23.712045618855058, 'pwr_req_dBm': 22.973, 'pwr_ext_dBm': 0.9223015226843518, 'rem_bits': 4712.0, 'mem_num': 71.0, 'eager_rate': 36.63437309374906}
# 	],
# 	[
# 		{'gid': 5.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 35.174129219912174, 'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002, 'rem_bits': 10712.0, 'mem_num': 17.0, 'eager_rate': 55.13946065977497},
# 		{'gid': 4.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 29.93949825941053, 'pwr_req_dBm': 22.959, 'pwr_ext_dBm': 2.729511581869497, 'rem_bits': 14792.0, 'mem_num': 10.0, 'eager_rate': 30.285257579987405},
# 		{'gid': 7.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 38.548690819308135, 'pwr_req_dBm': 22.955, 'pwr_ext_dBm': 3.1318014527925526, 'rem_bits': 3688.0, 'mem_num': 9.0, 'eager_rate': 87.80345203054216},
# 		{'gid': 2.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 24.740863098677366, 'pwr_req_dBm': 22.969, 'pwr_ext_dBm': 1.5202830452677776, 'rem_bits': 5640.0, 'mem_num': 9.0, 'eager_rate': 85.85339640344525},
# 		{'gid': 6.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 38.8170972756655, 'pwr_req_dBm': 22.955, 'pwr_ext_dBm': 3.1318014527925526, 'rem_bits': 7240.0, 'mem_num': 10.0, 'eager_rate': 36.47458385930194},
# 		{'gid': 8.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 28.355397019502817, 'pwr_req_dBm': 22.961, 'pwr_ext_dBm': 2.51331754988261, 'rem_bits': 216978.0, 'mem_num': 4.0, 'eager_rate': 0.605619464349728}
# 	]
# ]
# QoS_GP_CONF = [
# 	[
# 		{'gid': 2.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 18.2830342730702, 'pwr_req_dBm': 22.992, 'pwr_ext_dBm': -4.350942629063706, 'rem_bits': 290455.0, 'mem_num': 9.0, 'eager_rate': 3.337574931072202},
# 		{'gid': 5.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 18.52400268437595, 'pwr_req_dBm': 22.989, 'pwr_ext_dBm': -2.9694151005379332, 'rem_bits': 29088.0, 'mem_num': 7.0, 'eager_rate': 177.86494247807138},
# 		{'gid': 8.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 28.89612304419515, 'pwr_req_dBm': 22.96, 'pwr_ext_dBm': 2.6227721508302038, 'rem_bits': 14808.0, 'mem_num': 4.0, 'eager_rate': 28.062632286141262},
# 	]
# ]
# QoS_GP_CONF = [
# 	[
# 		{'gid': 4.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 32.07430265545245, 'pwr_req_dBm': 22.957, 'pwr_ext_dBm': 2.9353591822750817, 'rem_bits': 134283.0, 'mem_num': 8.0, 'eager_rate': 3.6125755646560775},
# 		{'gid': 2.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 31.393119329280523, 'pwr_req_dBm': 22.957, 'pwr_ext_dBm': 2.9353591822750817, 'rem_bits': 22643.0, 'mem_num': 10.0, 'eager_rate': 19.355847135066576},
# 		{'gid': 1.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 17.81152726123139, 'pwr_req_dBm': 22.999, 'pwr_ext_dBm': -13.37834310342348, 'rem_bits': 3082400.0, 'mem_num': 29.0, 'eager_rate': 86.84910822918255},
# 		{'gid': 5.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 35.645813350694, 'pwr_req_dBm': 22.955, 'pwr_ext_dBm': 3.1318014527925526, 'rem_bits': 288760.0, 'mem_num': 4.0, 'eager_rate': 0.24074566078393395}
# 	]
# ]
QoS_GP_CONF = [
    [
        {
            'gid': 8.0,
            'qos': 1.0,
            'rbf_w': 2.0,
            'rbf_h': 1.0,
            'sinr_max': 30.771152985394004,
            'pwr_req_dBm': 22.958,
            'pwr_ext_dBm': 2.833666714960843,
            'rem_bits': 10668.0,
            'mem_num': 7.0,
            'eager_rate': 5.736917255360139
        },
        {
            'gid': 4.0,
            'qos': 1.0,
            'rbf_w': 2.0,
            'rbf_h': 1.0,
            'sinr_max': 48.205159715810915,
            'pwr_req_dBm': 22.954,
            'pwr_ext_dBm': 3.2267555049165395,
            'rem_bits': 21400.0,
            'mem_num': 6.0,
            'eager_rate': 7.2555054815327855
        },
        {
            'gid': 3.0,
            'qos': 1.0,
            'rbf_w': 2.0,
            'rbf_h': 1.0,
            'sinr_max': 27.999893320750626,
            'pwr_req_dBm': 22.961,
            'pwr_ext_dBm': 2.51331754988261,
            'rem_bits': 22888.0,
            'mem_num': 8.0,
            'eager_rate': 17.234669657017825
        },
        {
            'gid': 6.0,
            'qos': 1.0,
            'rbf_w': 2.0,
            'rbf_h': 1.0,
            'sinr_max': 30.58000469893093,
            'pwr_req_dBm': 22.958,
            'pwr_ext_dBm': 2.833666714960843,
            'rem_bits': 2014639.0,
            'mem_num': 5.0,
            'eager_rate': 0.527111997358755
        },
        {
            'gid': 5.0,
            'qos': 1.0,
            'rbf_w': 2.0,
            'rbf_h': 1.0,
            'sinr_max': 35.46719559288359,
            'pwr_req_dBm': 22.955,
            'pwr_ext_dBm': 3.1318014527925526,
            'rem_bits': 940712.0,
            'mem_num': 2.0,
            'eager_rate': 0.26013374238008957
        },
        {
            'gid': 1.0,
            'qos': 1.0,
            'rbf_w': 2.0,
            'rbf_h': 1.0,
            'sinr_max': 15.017080245943824,
            'pwr_req_dBm': 22.969,
            'pwr_ext_dBm': 1.5202830452677776,
            'rem_bits': 80960.0,
            'mem_num': 46.0,
            'eager_rate': 11.839111397068704
        }
    ]
]

RES_CONF = {'rbf_h': 50.0, 'rbf_w': 2.0, 'max_pwr_dBm': 23.0}


In [6]:
out = io.StringIO()
alloc_report, exitflag = MATLAB_ENG.PlannerV1(RES_CONF, QoS_GP_CONF, nargout=2, stdout=out)
print(alloc_report)

{'g8': {'t0': {'c10': 24}}, 'g4': {'t0': {'c15': 23}}, 'g3': {'t0': {'c15': 25}}, 'g6': {'t0': {}}, 'g5': {'t0': {'c12': 26}}, 'g1': {'t0': {'c12': 2}}}


In [7]:
print(out.getvalue())

double
double
double
double
double
double
double
double
double
double
G8 - req_pwr: 22.958000dB/197.605942mW ext_pwr:2.833667dB/1.920289mW
G4 - req_pwr: 22.954000dB/197.424024mW ext_pwr:3.226756dB/2.102207mW
G3 - req_pwr: 22.961000dB/197.742491mW ext_pwr:2.513318dB/1.783741mW
G6 - req_pwr: 22.958000dB/197.605942mW ext_pwr:2.833667dB/1.920289mW
G5 - req_pwr: 22.955000dB/197.469488mW ext_pwr:3.131801dB/2.056744mW
G1 - req_pwr: 22.969000dB/198.107081mW ext_pwr:1.520283dB/1.419150mW
OMA Allocation Done!
max_ext_pwr:2.102207mW min_ext_pwr:1.419150mW
G8 - cqi_max:10 cqi_min:9 sinr_max_sdn:1194.305132 sinr_max:30.771153
G6 - cqi_max:10 cqi_min:9 sinr_max_sdn:1142.879571 sinr_max:30.580005
G5 - cqi_max:12 cqi_min:11 sinr_max_sdn:3521.434050 sinr_max:35.467196
G1 - cqi_max:2 cqi_min:1 sinr_max_sdn:31.747390 sinr_max:15.017080
NOMA Allocation Done!
===== QoS:1, Gid:8, Geo:(2,1) =====
{
 --- OMA Layer ---
 --- NOMA Layer ---
 cqi:10(x24), pwr:1.831828e+00 , rbs:[(5,1)(10,1)(11,1)(12,1)(13,1)(14,1

In [8]:
MATLAB_ENG.version()

'9.9.0.1467703 (R2020b)'

In [9]:
import matlab
print(matlab.__file__)

/home/OD/.local/share/virtualenvs/ProjectSumo-0-222X3X/lib/python3.8/site-packages/matlab/__init__.py


In [10]:
allocator = ResourceAllocatorNomaApprox(RES_CONF,QoS_GP_CONF)
allocator()

{'g3': {'t0': {'c15': 25}},
 'g1': {'t0': {'c12': 25, 'c1': 25}},
 'g4': {'t0': {'c15': 23}},
 'g8': {'t0': {'c9': 2}}}